<a href="https://colab.research.google.com/github/RCSnyder/Sales_Forecasting/blob/kishen-model/kishen_model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
#impoting libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import cross_val_score as cvs

In [18]:
#import dataset
train       = pd.read_csv('/content/drive/My Drive/CSCE 5214 Group #1 /sales_train.csv')
#test        = pd.read_csv('test.csv')
#submission  = pd.read_csv('sample_submission.csv')
items       = pd.read_csv('/content/drive/My Drive/CSCE 5214 Group #1 /items.csv')
#item_cats   = pd.read_csv('item_categories.csv')
#shops       = pd.read_csv('shops.csv')

In [19]:
#Down casts the data entries from int64 to int32 and float64 to float32
#This reduces the size of the records by almost half. (From 134mb to 61mb)
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

#%%
#Calls the downcasting function
train = downcast_dtypes(train)
items = downcast_dtypes(items)

In [20]:
# grouping for training model

# added the item_category into sales_train
merged2 = pd.merge(train, items[['item_id', 'item_category_id']], on = 'item_id')

# group data by month and shop_id, return sum of items sold per shop per month
# grouped by price as specials could result in higher sales
month_group2 = pd.DataFrame(merged2.groupby(['date_block_num', 'shop_id', 'item_category_id', 'item_id', 'item_price'])['item_cnt_day'].sum().reset_index())


In [21]:
# nominal intergers can not be converted to binary encoding, convert to string
month_group2['date_block_num'] = [('month ' + str(i)) for i in month_group2['date_block_num']]
month_group2['shop_id'] = [('shop ' + str(i)) for i in month_group2['shop_id']]
month_group2['item_category_id'] = [('item_category ' + str(i)) for i in month_group2['item_category_id']]
month_group2['item_id'] = [('item ' + str(i)) for i in month_group2['item_id']]

In [22]:
month_group2

,date_block_num,shop_id,item_category_id,item_id,item_price,item_cnt_day
0,month 0,shop 0,item_category 2,item 5572,1322.0,10.0
1,month 0,shop 0,item_category 2,item 5573,560.0,1.0
2,month 0,shop 0,item_category 2,item 5575,806.0,4.0
3,month 0,shop 0,item_category 2,item 5576,2231.0,5.0
4,month 0,shop 0,item_category 2,item 5609,2381.0,1.0
...,...,...,...,...,...,...
1739017,month 33,shop 59,item_category 79,item 17717,1250.0,4.0
1739018,month 33,shop 59,item_category 79,item 17717,1999.0,1.0
1739019,month 33,shop 59,item_category 83,item 22087,119.0,6.0
1739020,month 33,shop 59,item_category 83,item 22088,119.0,2.0


In [9]:
# break into X and Y, where X is the inputs and Y is our output
X = month_group2.iloc[:, :-1].values
Y = month_group2.iloc[:,-1].values

In [10]:
# Encoding categorical data
# used to provide a value to data that can be then used in equations, eg. Friday = 1 etc.
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# the only variable which is not categorical is item_price, hence all other variable will be converted
ct = ColumnTransformer([('encoder', OneHotEncoder(), [0, 1, 2, 3])], remainder = 'passthrough')
#X = np.array(ct.fit_transform(X), dtype=object)
X = ct.fit_transform(X)
# convert back to 2-D representation of the matrix from sparse matrix representation 
#X = X.todense()

In [11]:
# split data set into training set and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size = 0.2, random_state = 0)

In [ ]:
# fitting multiple linear regression to the training set
from sklearn.linear_model import LinearRegression
regressor = LinearRegression() 
regressor.fit(X_train, Y_train)

In [ ]:
# Cross validation
from sklearn.model_selection import cross_val_score as cvs
reg_score = cvs(regressor, X_train, Y_train, scoring = "neg_mean_squared_error", cv = 10)
reg_score = np.sqrt(-reg_score)

In [12]:
def display_scores(scores):
    print("rmse Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [15]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=1e-2, fit_intercept=False, solver="cholesky")
#ridge_reg = Ridge(alpha)
ridge_reg.fit(X_train, Y_train)

Ridge(alpha=0.01, copy_X=True, fit_intercept=False, max_iter=None,
      normalize=False, random_state=None, solver='cholesky', tol=0.001)

In [16]:
ridge_score = cvs(ridge_reg, X_train, Y_train, scoring = "neg_mean_squared_error", cv = 5)
ridge_score = np.sqrt(-ridge_score)
#display_scores(reg_score)
display_scores(ridge_score)

rmse Scores: [5.84978032 5.80009775 6.67136631 6.69919088 7.91456632]
Mean: 6.587000316114688
Standard deviation: 0.7674431344051948


In [13]:
# Grid search to find good hyperparameters
from sklearn.model_selection import GridSearchCV

# get parameters for hyperparameter tuning
#print(regressor.get_params().keys())
print(ridge_reg.get_params().keys())


#param_grid_reg = [{'fit_intercept': [True, False], 'normalize': [True, False]}]

param_grid_ridge = [{'alpha': [1e-1, 1]
                     , 'fit_intercept': [False]
                     , 'normalize': [True, False]
                     , 'solver': ['cholesky', 'lsqr']}]

#grid_search_reg = GridSearchCV(regressor, param_grid_reg, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search_ridge = GridSearchCV(ridge_reg, param_grid_ridge, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

#grid_search_reg.fit(X_test, Y_test)
grid_search_ridge.fit(X_test, Y_test)

dict_keys(['alpha', 'copy_X', 'fit_intercept', 'max_iter', 'normalize', 'random_state', 'solver', 'tol'])


KeyboardInterrupt: ignored

In [ ]:
cvres = grid_search_ridge.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)